In [1]:
import pandas as pd
from sklearn import model_selection
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.pipeline import Pipeline
import pickle

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import RegexpTokenizer

import re
import string

import nest_asyncio
import twint
import pandas as pd
nest_asyncio.apply()
c = twint.Config()

#Instantiating NLTK Stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

new_stopwords = ['hurricane', 'harvey', 'irma', 'matthew', 'maria', 'hurricaneharvey', 'hurricanemaria', 'hurricaneirma']
punctuations = string.punctuation
stopwords.extend(new_stopwords)
stopwords.extend(punctuations)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brettzimmerman/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#from models import tokenize, remove_junk, predict

In [3]:
#Creating a function to tokenize text
def tokenize(text):
    regex_token = RegexpTokenizer(r"([a-zA-Z]+(?:’[a-z]+)?)")
    tokenized_tweet = regex_token.tokenize(text)
    tweet_tokens = [word.lower() for word in tokenized_tweet]
    return tweet_tokens

In [4]:
#Creating a function to clean the Tweets with Regex
def remove_junk(text):
    text = re.sub(r'@[A-Za-z0-9#]+', '', str(text)) #remove @mentions 
    text = re.sub(r'RT[\s]+', '', str(text)) # remove RT
    text = re.sub(r'https?:\/\/\S+', '', text) # removes actual links
    text = re.sub(r'#', '', str(text)) # remove hashtag symbol
    return text

In [5]:
#Creating a function that will apply the model to predict text
def predict(pipeline, text):
    # Predict the sentiment
    #textdata = pipeline.transform()
    text = [remove_junk(i) for i in text]
    sentiment = pipeline.predict(text)
    
    # Make a list of text with sentiment.
    data = []
    for text, pred in zip(text, sentiment):
        data.append((text,pred))
        
    # Convert the list into a Pandas DataFrame.
    df = pd.DataFrame(data, columns = ['text','sentiment'])
    df = df.replace([0,1,2,3], ['danger', 'emergency', 'other', 'relief'])
    return df

In [6]:
def twint_to_pd(columns):
    return twint.output.panda.Tweets_df[columns]

In [7]:
c.Search= 'hurricane'
c.Lang= 'en'
c.Pandas= True
c.Limit= 100
c.Since = '2021-8-25'
c.Until = '2021-9-04'
twint.run.Search(c)

1433580493086339073 2021-09-02 19:59:58 -0400 <ragusajr161> @SenTuberville Well, if the United States hadn't abused the Banana Republics for the last two centuries and usurped all their Natural Resources for Corporate profit and then Doused them with Climate Chaos and Multiple Major Hurricanes, we many not have this problem. Imperialism Kills!
1433580492918599681 2021-09-02 19:59:58 -0400 <KevinMyattWx> Retweeting this article just to make it clear that Fujita Scale is for tornadoes &amp; Saffir-Simpson is for hurricanes. Made a mental mistake in 1st version of article, now corrected. Exhausting week of hurricane &amp; tornadoes got to me I guess.
1433580476267220992 2021-09-02 19:59:54 -0400 <CIArelief> Update from TN after Hurricane Ida 🌀 #HurricaneIda #Disasterrelief #volunteers #bringinghope #redshirts #waverlyfloods-  https://t.co/EvfWm4Ak9R
1433580461364760578 2021-09-02 19:59:51 -0400 <mikeOS_X> Hurricane basement water, famcovid (it’s fine), work week from hell, and mower probl

In [8]:
data = twint_to_pd(['tweet'])
test_list = data['tweet'].tolist()

In [9]:
# Load model of choice: naive_bayes, logistic_regression, roberta
with open('naive_bayes.pickle', 'rb') as file:
    classifier = pickle.load(file)
sentiment = predict(classifier, test_list)
print(sentiment)

                                                 text sentiment
0    Well, if the United States hadn't abused the ...    danger
1   Retweeting this article just to make it clear ...     other
2   Update from TN after Hurricane Ida 🌀 Hurricane...    relief
3   Hurricane basement water, famcovid (it’s fine)...     other
4   Dozens arrested for looting in New Orleans aft...     other
..                                                ...       ...
74  It had been looking like a low key post hurric...     other
75  2yo NYC boy among 40+ killed after Hurricane I...    danger
76  I was banned during Hurricane Ida for going li...    danger
77     He clearly was on holiday during the massiv...    danger
78  to become powerful Category 4 storm   David an...    danger

[79 rows x 2 columns]


In [10]:
relief = sentiment.loc[sentiment['sentiment'] == 'relief']
relief['text'].to_csv('relief.csv', index=False)

danger = sentiment.loc[sentiment['sentiment'] == 'danger']
danger['text'].to_csv('danger.csv', index=False)

emergency = sentiment.loc[sentiment['sentiment'] == 'emergency']
emergency['text'].to_csv('emergency.csv', index=False)

other = sentiment.loc[sentiment['sentiment'] == 'other']
other['text'].to_csv('other.csv', index=False)